# Interview Prep Chatbot

## Linked with OpenAI API KEY

In [1]:
%load_ext dotenv
%dotenv

## Loading Below libraries

In [2]:
# Loading libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from IPython.display import clear_output

## Setting Global Variables

In [3]:
# GLOBAL VARIABLES
TOPIC = ""
USER_QUESTION = ""
USER_ANSWER = ""
USER_RATING = ""

## Setting the Evaluation Martix Class for Answer Evaluation

In [4]:
# Evaluation class matrix definition
class Evaluation(BaseModel):
    core_understanding: int = Field(description="Score out of 5")
    practical_understanding: int = Field(description="Score out of 5")
    techincal_precision: int = Field(description="Score out of 5")
    common_pitfall: int = Field(description="Score out of 5")

pydantic_parser = PydanticOutputParser(pydantic_object=Evaluation)

## Prompt, Model and Parser Settings

In [5]:
# Question Prompt Template definition
question_prompt_template = PromptTemplate.from_template('''You are an interviewer.
Ask one technical interview question on the topic {topic}.''')

# Evaluation Prompt Template definition
evaluation_prompt = ChatPromptTemplate.from_template("""
Evaluate the answer strictly and return JSON.

{format_instructions}

Question: {question}
Answer: {answer}
""").partial(format_instructions=pydantic_parser.get_format_instructions())

# Chat Model definition
chat = ChatOpenAI(
    model='gpt-4',
    temperature=0,
    seed=25
)

# Output Parser definition
str__parser = StrOutputParser()

## Function Definitions

### 1. `get_topic()` function to get the topic from the user

In [6]:
def get_topic():
    global TOPIC
    while True:
        user_input = input("Enter your topic").lower().strip()
        if user_input and user_input != 'exit':
            TOPIC = user_input
            return 0

### 2. `generate_question()` Function

The `generate_question()` function is used to generate a question based on the user-provided topic.

#### Arguments

- **`llm`**  
  Chat model used to generate the question.

- **`prompt`**  
  Prompt template passed to the chat model to guide question generation.

- **`parser`**  
  Parser that extracts the important text from the model response without any metadata.

- **`topic`**  
  User-defined topic on which the question will be based.


In [7]:
def generate_question(llm, prompt, parser, topic):
    global USER_QUESTION
    chain = prompt | llm | parser
    question = chain.invoke(topic)
    USER_QUESTION = question

### 2. `get_user_answer()` Function

The `get_user_answer()` function allows the user to enter the answer of the question

#### Arguments

- **`question`**  
  Question generated by the chat model.


In [8]:
def get_user_answer(question):
    global USER_ANSWER
    print(f"Your Question is: {question}")
    user_answer = input("Enter your answer").lower()
    USER_ANSWER = user_answer

### 2. `show_rating()` Function

The `show_rating()` function will print the evalution in nice format

In [9]:
def show_rating():
    print(f"Your Question: {USER_QUESTION}")
    print('-'*10)
    print(f"Your Answer: {USER_ANSWER.capitalize()}")
    print('-'*10)
    for i in USER_RATING:
        print(f"{' '.join(i[0].split('_')).capitalize()}: {'⭐'*i[1]} ({i[1]}/5)")

### 2. `eval_resonse()` Function

The `eval_resonse()` function is used to evalute the user answer and model generated question using the chat model.

#### Arguments

- **`llm`**  
  Chat model used to generate the question.

- **`prompt`**  
  Prompt template passed to the chat model to guide question generation.

- **`parser`**  
  Parser that extracts the important text from the model response without any metadata.

- **`question`**  
  Question generated by the model

- **`answer`**  
  Answer entered by the user


In [10]:
def eval_resonse(llm, prompt, parser, question, answer):
    global USER_RATING
    chain = prompt | llm | parser
    result = chain.invoke({
    'question': question,
    'answer' : answer
    })

    USER_RATING = result
        
    print("Your Answer has been successfully Evaluated ✅")
    show_rating()

### 2. `start_chatbot()` Function

The `start_chatbot()` function is used to start the chatbot.

In [11]:
def start_chatbot():
    get_topic()
    generate_question(llm=chat, prompt=question_prompt_template, parser=str__parser, topic=TOPIC)
    clear_output()
    answer = get_user_answer(question=USER_QUESTION)
    clear_output()
    eval_resonse(llm=chat, prompt=evaluation_prompt, parser=pydantic_parser, question=USER_QUESTION, answer=USER_ANSWER)

In [12]:
start_chatbot()

Your Answer has been successfully Evaluated ✅
Your Question: Can you explain how Langchain utilizes OpenAI's GPT-3 model to generate human-like text and what potential applications this technology might have?
----------
Your Answer: Langchain integrates openai’s gpt-3 by structuring prompts, chaining model calls, and managing context to generate coherent, human-like text. it enables applications such as chatbots, document question answering, automated content generation, interview simulators, agents, and workflow automation powered by large language models.
----------
Core understanding: ⭐⭐⭐⭐⭐ (5/5)
Practical understanding: ⭐⭐⭐⭐⭐ (5/5)
Techincal precision: ⭐⭐⭐⭐⭐ (5/5)
Common pitfall: ⭐⭐⭐⭐⭐ (5/5)
